<a href="https://colab.research.google.com/github/muzammil-max/ML-and-DL-Journey/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifiers
They are used to seperate the data points to different classes in a seperate labels

In [ ]:
!pip install scikit-learn

In [29]:
!pip install tensorflow==2.15

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

In [3]:
CSV_COLUMNS_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


In [4]:
train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")


train= pd.read_csv(train_path, names=CSV_COLUMNS_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMNS_NAMES, header=0)

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [5]:
if 'Species' in train.columns:
    train_y = train.pop('Species')
if 'Species' in test.columns:
    test_y = test.pop('Species')

train.head() #the species column is now gone



,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [6]:
train.shape

(120, 4)

In [7]:
#now make input function

def input_fn(features, labels, training=True, batch_size=256):
    #convert the inputs to a dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

In [8]:
feature_columns = []
for key in train.keys():
    feature_columns.append(tf.feature_column.numeric_column(key=key))
print(feature_columns)

Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [15]:
#Building the model!
#there are two types of model
#Linear classifier and DNN classifier (deep neural network)
#DNN is best choice because in many datasets we may not find linear relation


# This code is building a Deep Neural Network (DNN) Classifier using TensorFlow's Estimator API.
# This type of model is great for finding complex, non-linear relationships in your data.
# Here's what each part does:
#   - tf.estimator.DNNClassifier(...): This is the main function that creates the DNN classifier object.
#   - feature_columns=feature_columns: This tells the model what your input data looks like. The feature_columns variable you created earlier defines each of the input features (SepalLength, SepalWidth, PetalLength, PetalWidth) as numeric columns.
#   - hidden_units=[30, 10]: This defines the architecture of your neural network. It's saying:
#       - There are two hidden layers.
#       - The first hidden layer has 30 neurons.
#       - The second hidden layer has 10 neurons.
#       - The number of neurons in each layer is a hyperparameter you can tune to improve your model's performance.
#   - n_classes=3: This specifies the number of possible output classes. In the Iris dataset, there are three species of iris flowers (Setosa, Versicolor, and Virginica), so we set this to 3.
classifier = tf.estimator.DNNClassifier(
    feature_columns = feature_columns,
    hidden_units = [30, 10],
    n_classes = 3
)

### Training the Model

This code is training the `DNNClassifier` model that you created in the previous step. Here's a breakdown of what's happening:

*   **`classifier.train(...)`**: This is the method that starts the training process for your model.

*   **`input_fn=lambda: input_fn(train, train_y, training=True)`**: This is the most important part. It tells the `train` method how to get the training data.
    *   `input_fn`: This is the argument that expects a function that provides the training data.
    *   `lambda: ...`: This creates a simple, anonymous function.
    *   `input_fn(train, train_y, training=True)`: This calls the `input_fn` function you defined earlier.
        *   `train`: This is your training data (the features).
        *   `train_y`: This is your training labels (the correct answers).
        *   `training=True`: This tells the `input_fn` that you are in training mode, so it should shuffle the data and repeat it.

*   **`steps=5000`**: This tells the training process to run for 5000 steps. In each step, the model will process a batch of data, calculate the error, and adjust its internal weights to get better at making predictions.

### Why use a `lambda` function?

The `input_fn` argument of the `classifier.train()` method has a specific requirement: **it needs a function that takes no arguments.**

However, the `input_fn` function that you defined earlier *does* take arguments: `features`, `labels`, `training`, and `batch_size`.

So, how do you pass a function that requires arguments to a method that expects a function with no arguments?

This is where `lambda` comes in.

A `lambda` function is a small, anonymous function. It's a way to create a function on the fly without giving it a name. In this case, the `lambda` function is used to "wrap" your `input_fn` and create a new function that takes no arguments.

In [32]:
import logging

In [36]:
tf.get_logger().setLevel(logging.INFO)

In [37]:
classifier.train(
    input_fn = lambda: input_fn(train, train_y, training=True),
    steps = 5000
)



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpemflqe0v/model.ckpt-35000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 35000...
INFO:tensorflow:Saving checkpoints for 35000 into /tmp/tmpemflqe0v/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 35000...
INFO:tensorflow:loss = 0.15159653, step = 35000
INFO:tensorflow:global_step/sec: 307.983
INFO:tensorflow:loss = 0.14808276, step = 35100 (0.328 sec)
INFO:tensorflow:global_step/sec: 586.513
INFO:tensorflow:loss = 0.15344876, step = 35200 (0.168 sec)
INFO:tensorflow:global_step/sec: 583.354
INFO:tensorflow:loss = 0.15551573, step = 35300 (0.172 sec)
INFO:tensorflow:global_step/sec: 565.715
INFO:tensorflow:loss = 0.14857325, step = 35400 (0.178 sec)